In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        import os


import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import RMSprop
from torch.multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

from torchvision import transforms

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import STL10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [ ]:
import keras

with open('/kaggle/input/stl10-binary-files/stl10_binary/train_X.bin', 'rb') as f:
        everything = np.fromfile(f, dtype=np.uint8)

        X_train = np.reshape(everything, (-1, 96, 96, 3))
       # X_test = np.transpose(X_test, (0, 3, 2, 1))

with open('/kaggle/input/stl10-binary-files/stl10_binary/train_y.bin', 'rb') as f:
        Y_train = np.fromfile(f,dtype=np.uint8)
with open('/kaggle/input/train-label/train_label.npy', 'rb') as f:
        Y1_train = np.fromfile(f)        
with open('/kaggle/input/stl10-binary-files/stl10_binary/test_X.bin', 'rb') as f:
        everything = np.fromfile(f, dtype=np.uint8)

        X_test = np.reshape(everything, (-1, 96, 96, 3))
       # X_test = np.transpose(X_test, (0, 3, 2, 1))

with open('/kaggle/input/stl10-binary-files/stl10_binary/test_y.bin', 'rb') as f:
        Y_test = np.fromfile(f,dtype=np.uint8)
with open('/kaggle/input/stl10-binary-files/stl10_binary/unlabeled_X.bin', 'rb') as f:
        everything = np.fromfile(f, dtype=np.uint8)

        unlabelled_images = np.reshape(everything, (-1, 96, 96, 3))
       # unlabelled_images = np.transpose(unlabelled_images, (0, 3, 2, 1))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization


In [ ]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims
X_train = preprocess_image_input(X_train)
X_test = preprocess_image_input(X_test)



In [ ]:

inputs = tf.keras.layers.Input(shape=(96, 96,3)  )
resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    

resnet_feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(96, 96, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
classification_output = tf.keras.layers.GlobalAveragePooling2D()(resnet_feature_extractor)
classification_output = tf.keras.layers.Flatten()(classification_output)
classification_output = tf.keras.layers.Dense(1024, activation="relu")(classification_output)
classification_output = tf.keras.layers.Dense(512, activation="relu")(classification_output)
classification_output = tf.keras.layers.Dense(11, activation="softmax", name="classification")(classification_output) 
model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

In [ ]:


X_test.shape


In [ ]:

X_train.shape

In [ ]:
EPOCHS = 10
history = model.fit(X_train,Y_train, epochs=EPOCHS, validation_data = (X_test,Y_test), batch_size=64)

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test, batch_size=64)

In [ ]:
sd=model.predict(X_test)